In [1]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "RIGHT_WRIST",
    "LEFT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [3]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [4]:
# load dataset
df = describe_dataset("./train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 979 
Number of columns: 37

Labels: 
C    557
L    422
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [5]:
with open("./model/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [6]:
# Standard Scaling of features
x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

In [15]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

# Final Results
final_models = {}

In [9]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [10]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [11]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo',
)
tuner_3l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 02s]
val_accuracy: 0.9948979616165161

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 01m 06s
INFO:tensorflow:Oracle triggered exit


In [12]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x00000208FEB779D0>
Layer-2: 448 units, func:  <function relu at 0x00000208FEB779D0>
Layer-3: 2 units, func:  <function softmax at 0x00000208FEB70F70>

Other params:
learning_rate: 0.01
Epoch 1/100
79/79 [==============================] - 1s 5ms/step - loss: 0.0872 - accuracy: 0.9668 - val_loss: 4.8449e-04 - val_accuracy: 1.0000
Epoch 2/100
79/79 [==============================] - 0s 3ms/step - loss: 0.0140 - accuracy: 0.9949 - val_loss: 0.0039 - val_accuracy: 1.0000
Epoch 3/100
79/79 [==============================] - 0s 4ms/step - loss: 2.4864e-04 - accuracy: 1.0000 - val_loss: 1.0227e-04 - val_accuracy: 1.0000
Epoch 4/100
79/79 [==============================] - 0s 3ms/step - loss: 3.8161e-05 - accuracy: 1.0000 - val_loss: 7.1878e-05 - val_accuracy: 1.0000
Epoch 5/100
79/79 [==============================] - 0s 3ms/step - loss: 2.4436e-05 - accuracy: 1.0000 - val_loss: 5.5778e-05 - val_accuracy: 1.0000
Epoch 6/

79/79 [==============================] - 0s 3ms/step - loss: 2.1558e-07 - accuracy: 1.0000 - val_loss: 3.0114e-06 - val_accuracy: 1.0000
Epoch 55/100
79/79 [==============================] - 0s 3ms/step - loss: 2.0477e-07 - accuracy: 1.0000 - val_loss: 2.9530e-06 - val_accuracy: 1.0000
Epoch 56/100
79/79 [==============================] - 0s 3ms/step - loss: 1.9335e-07 - accuracy: 1.0000 - val_loss: 2.8879e-06 - val_accuracy: 1.0000
Epoch 57/100
79/79 [==============================] - 0s 3ms/step - loss: 1.8239e-07 - accuracy: 1.0000 - val_loss: 2.8065e-06 - val_accuracy: 1.0000
Epoch 58/100
79/79 [==============================] - 0s 3ms/step - loss: 1.7417e-07 - accuracy: 1.0000 - val_loss: 2.7499e-06 - val_accuracy: 1.0000
Epoch 59/100
79/79 [==============================] - 0s 3ms/step - loss: 1.6503e-07 - accuracy: 1.0000 - val_loss: 2.6934e-06 - val_accuracy: 1.0000
Epoch 60/100
79/79 [==============================] - 0s 3ms/step - loss: 1.5666e-07 - accuracy: 1.0000 - val_los

In [16]:
final_models["3_layers"] = model_3l


In [17]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [18]:
import keras
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_2'
)

tuner_5l.search(
    x_train, 
    y_train, 
    epochs=10, 
    callbacks=[keras.callbacks.EarlyStopping(patience=5)]
)

Trial 30 Complete [00h 00m 02s]
accuracy: 1.0

Best accuracy So Far: 1.0
Total elapsed time: 00h 01m 05s
INFO:tensorflow:Oracle triggered exit


In [19]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x00000208FEB779D0>
Layer-2: 256 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-3: 320 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-4: 480 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-5: 2 units, func:  <function softmax at 0x00000208FEB70F70>

Other params:
learning_rate: 0.001
Epoch 1/100
79/79 [==============================] - 1s 10ms/step - loss: 0.1240 - accuracy: 0.9566 - val_loss: 0.3868 - val_accuracy: 0.9388
Epoch 2/100
79/79 [==============================] - 0s 6ms/step - loss: 0.0675 - accuracy: 0.9821 - val_loss: 0.0144 - val_accuracy: 0.9949
Epoch 3/100
79/79 [==============================] - 0s 5ms/step - loss: 0.0050 - accuracy: 0.9974 - val_loss: 4.6069e-05 - val_accuracy: 1.0000
Epoch 4/100
79/79 [==============================] - 0s 5ms/step - loss: 6.6957e-05 - accuracy: 1.0000 - val_loss: 2.9244e-05 - val_accuracy: 1.0000
Epoch 5/100
79/79 [==============

In [20]:
final_models["5_layers"] = model_5l


In [29]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [30]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_3'
)
tuner_7lD.search(x_train, y_train, epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=5)])

INFO:tensorflow:Reloading Tuner from keras_tuner_dir\keras_tuner_demo_3\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [31]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x00000208FEB779D0>
Layer-2: 448 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-3: 0 units, func: None
Layer-4: 192 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-5: 0 units, func: None
Layer-6: 128 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-7: 2 units, func:  <function softmax at 0x00000208FEB70F70>

Other params:
dropout_1: 0.30000000000000004
dropout_2: 0.1
learning_rate: 0.001
Epoch 1/100
79/79 [==============================] - 1s 8ms/step - loss: 0.1802 - accuracy: 0.9195 - val_loss: 0.0290 - val_accuracy: 0.9898
Epoch 2/100
79/79 [==============================] - 0s 5ms/step - loss: 0.0177 - accuracy: 0.9936 - val_loss: 0.0324 - val_accuracy: 0.9949
Epoch 3/100
79/79 [==============================] - 0s 4ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0322 - val_accuracy: 0.9949
Epoch 4/100
79/79 [==============================] - 0s 4ms/step - loss: 5.4264e-04

In [32]:
final_models["7_layers_with_dropout"] = model_7lD

In [33]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [34]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_6'
)
tuner_7l.search(x_train, y_train, epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=5)])

Trial 30 Complete [00h 00m 03s]
accuracy: 1.0

Best accuracy So Far: 1.0
Total elapsed time: 00h 01m 12s
INFO:tensorflow:Oracle triggered exit


In [35]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x00000208FEB779D0>
Layer-2: 448 units, func:  <function relu at 0x00000208FEB779D0>
Layer-3: 96 units, func:  <function relu at 0x00000208FEB779D0>
Layer-4: 320 units, func:  <function relu at 0x00000208FEB779D0>
Layer-5: 256 units, func:  <function relu at 0x00000208FEB779D0>
Layer-6: 256 units, func:  <function relu at 0x00000208FEB779D0>
Layer-7: 2 units, func:  <function softmax at 0x00000208FEB70F70>

Other params:
learning_rate: 0.001
Epoch 1/100
79/79 [==============================] - 1s 8ms/step - loss: 0.1056 - accuracy: 0.9604 - val_loss: 4.3986e-04 - val_accuracy: 1.0000
Epoch 2/100
79/79 [==============================] - 0s 5ms/step - loss: 0.0787 - accuracy: 0.9898 - val_loss: 0.0058 - val_accuracy: 0.9949
Epoch 3/100
79/79 [==============================] - 0s 4ms/step - loss: 0.0107 - accuracy: 0.9936 - val_loss: 0.0392 - val_accuracy: 0.9949
Epoch 4/100
79/79 [==============================] - 0

In [36]:
final_models["7_layers"] = model_7l

In [37]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x00000208FEB779D0>
Layer-2: 448 units, func:  <function relu at 0x00000208FEB779D0>
Layer-3: 2 units, func:  <function softmax at 0x00000208FEB70F70>

5_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x00000208FEB779D0>
Layer-2: 256 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-3: 320 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-4: 480 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-5: 2 units, func:  <function softmax at 0x00000208FEB70F70>

7_layers_with_dropout: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x00000208FEB779D0>
Layer-2: 448 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-3: 0 units, func: None
Layer-4: 192 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-5: 0 units, func: None
Layer-6: 128 units, func:  <function tanh at 0x00000208FEB77D30>
Layer-7: 2 units, func:  <funct

In [38]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[[113, 0], [0, 83]]"
1,5_layers,"[1.0, 1.0]","[1.0, 1.0]","[1.0, 1.0]","[[113, 0], [0, 83]]"
2,7_layers_with_dropout,"[0.991, 1.0]","[1.0, 0.988]","[0.996, 0.994]","[[113, 0], [1, 82]]"
3,7_layers,"[0.991, 1.0]","[1.0, 0.988]","[0.996, 0.994]","[[113, 0], [1, 82]]"


In [39]:
# load dataset
test_df = describe_dataset("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 419 
Number of columns: 37

Labels: 
L    269
C    150
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [40]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [41]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: k[1] + k[2] + k[3], reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout,"[0.629, 0.827]","[0.713, 0.766]","[0.669, 0.795]","[[107, 43], [63, 206]]"
1,7_layers,"[0.625, 0.905]","[0.867, 0.71]","[0.726, 0.796]","[[130, 20], [78, 191]]"
2,5_layers,"[0.58, 0.897]","[0.867, 0.651]","[0.695, 0.754]","[[130, 20], [94, 175]]"
3,3_layers,"[0.576, 0.756]","[0.553, 0.773]","[0.565, 0.765]","[[83, 67], [61, 208]]"


In [42]:
final_models["7_layers"].save("C:/Users/Alrowad/Exercise-correction/biceps_model/model/bicep_dp.h5")

In [43]:
for model_name, model in final_models.items():
    model.save(f"C:/Users/Alrowad/Exercise-correction/biceps_model/model/{model_name}.h5")